In [1]:
import pyomo.environ as pe
import pyomo.opt as po

### Problema Sequencing Tasks

Cristina Hernández, Macarena Vargas, Guillermo Ruiz, Beatriz Jiménez. 3ºIMAT B 

In [2]:
model = pe.ConcreteModel()

#### SETS
$t$: tasks {T1, T2, T3, T4, T5}

$m$: machines {m1, m2}


In [3]:
tasks = ['T1', 'T2', 'T3', 'T4', 'T5']
machines = ['m1', 'm2']
model.t = pe.Set(initialize=tasks, ordered = True)
model.m = pe.Set(initialize=machines, ordered = True)

# le voy a añadir un nuevo SET que coge todas las combinaciones de tareas
# para despues poder definir la variable t. 
model.pairs = pe.Set(initialize=[(i, j) for i in tasks for j in tasks if i != j])



#### PARAMETERS 

$T_t,m$: processing time of task t in machine m [h]

$D_t$: delivery time of task t [h]

In [4]:
T_dict = {('T1','m1'): 9, ('T1','m2'): 12,
            ('T2','m1'): 6, ('T2','m2'): 8,
            ('T3','m1'): 10, ('T3','m2'): 7,
            ('T4','m1'): 8, ('T4','m2'): 10,
            ('T5','m1'): 7, ('T5','m2'): 9} 
D_dict = {'T1': 40, 'T2': 25, 'T3': 33, 'T4': 45, 'T5': 45} 
model.T = pe.Param(model.t, model.m, initialize=T_dict)
model.D = pe.Param(model.t, initialize=D_dict)
 

### VARIABLES
$s_{t,m}$: starting time of task t in machine m [h]

$x_{t,t,m}$: task t done before task t' in machine m {0,1}

$p_{t}$ : delay of task t [h] 

$mx$ : maximum delay of all tasks 

$end$ : maximum delivery date of all tasts 


In [5]:

model.S = pe.Var(model.t, model.m, within = pe.NonNegativeIntegers)
model.X = pe.Var(model.pairs, model.m, within=pe.Binary) # le aplico el "pairs" para controlar t y t'
model.P = pe.Var(model.t , within =  pe.NonNegativeIntegers)
model.MX = pe.Var(within = pe.NonNegativeIntegers)
model.END = pe.Var(within = pe.NonNegativeIntegers)



#### OBJECTIVE FUNCTION
min $mx$ or min $end$

In [6]:

model.obj = pe.Objective(expr=model.MX, sense=pe.minimize) 
#model.obj_2 = pe.Objective(expr=model.END, sense=pe.minimize)


#### CONTRAINTS 

1. Task t in machine M2 starts only after finishing M1 


$S_{t,m=1}+ T_{t,m=1} \leq S_{t,m=2}   \forall t $


In [7]:
def m2_after_m1(model, t):
    return model.S[t, 'm1'] + model.T[t, 'm1'] <= model.S[t, 'm2']

model.capacidad = pe.Constraint(model.t, rule=m2_after_m1)


2. Task t and task t' cannot overlap in some machine m [h]

(dor every pair of task t and t', one must finish before the other)
$x_t,t',m$ = 1 -> t goes before t' 
$x_t,t',m$ = 0 -> t' goes before t


IF $x_{t,t',m}$ = 1 -> $S_{t,m}+ T_{t,m} \leq S_{t',m}  \forall t!=t , m $

that can be re-written as: 

 $S_{t,m}+ T_{t,m} - S_{t',m} \leq M *(1 - x_{t,t',m}) \forall t!=t , m $ 


In [8]:
M = sum(T_dict.values()) + 1  # big M as the sum of all processing times +1 

def sequencing_rule(model, t1, t2, m):
    return model.S[t1, m] + model.T[t1, m] <= model.S[t2, m] + M * (1 - model.X[t1, t2, m])
model.sequencing = pe.Constraint(model.pairs, model.m, rule=sequencing_rule)

3. Simetric processes : task t proceeds task t' or viceversa in machine m 


$x_{t,t',m}  + x_{t',t,m} = 1            \forall t!=t , m $

In [9]:
def order_symmetry(mdl, t1, t2, m):
    return mdl.X[t1, t2, m] + mdl.X[t2, t1, m] == 1
model.order_symmetry = pe.Constraint(model.pairs, model.m, rule=order_symmetry)


4. Delay time of each task t 


$S_{t,m = M2}  + T_{t,m=M2} - D_{t} \leq P_{t}            \forall t$

In [10]:
def delay_time_rule(model, t):
    return model.S[t, 'm2'] + model.T[t, 'm2'] - model.D[t] <= model.P[t]
model.delay_time = pe.Constraint(model.t, rule=delay_time_rule)

5. Task 3 needs to be processed before task 2

$S_{t=T3,m}  + T_{t=T3,m} \leq S_{t=2, m}            \forall m$

In [11]:
def task3_before_task2(model, m):
    return model.S['T3', m] + model.T['T3', m] <= model.S['T2', m]
model.task3_before_task2 = pe.Constraint(model.m, rule=task3_before_task2)

6. Maximum delay of all tasks 

$ MX \;\geq\; P_t \forall t$ 

In [12]:
def max_delay_time(model, t):
    return model.MX >= model.P[t]
model.delay_time_rule = pe.Constraint(model.t, rule=max_delay_time)

7. Maximum delivery date of lal tasks 

$ END \;\geq\; S_{t,m=M2} + T_{t,m=M2} \forall t$ 


In [13]:
def max_delivery_time(model, t):
    return model.END >= model.S[t, 'm2'] + model.T[t, 'm2']
model.max_delivery_time = pe.Constraint(model.t, rule=max_delivery_time)




8. Positive variables 
$ S_{t,m} , P_{t} \;\geq\; 0 $
(already declared in the definition of variables: within pe.NonNegativeIntegers)

In [14]:
def same_sequence(model, t1, t2):
    return model.X[t1, t2, 'm1'] == model.X[t1, t2, 'm2']
model.same_sequence = pe.Constraint(model.pairs, rule=same_sequence)


#### Solving problem 1 : 

In [15]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee=True) 

print("Status:", result.solver.status)
print("Termination:", result.solver.termination_condition)

Set parameter Username
Set parameter LicenseID to value 2707518
Academic license - for non-commercial use only - expires 2026-09-11
Read LP format model from file C:\Users\Guill\AppData\Local\Temp\tmpw0meb72i.pyomo.lp
Reading time = 0.05 seconds
x1: 122 rows, 57 columns, 284 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 122 rows, 57 columns and 284 nonzeros
Model fingerprint: 0xae0562de
Variable types: 0 continuous, 57 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 23.0000000
Presolve removed 72 rows and 37 columns
Presolve time: 0.01s
Presolved: 50 rows, 20 columns, 136 nonzeros
Variable types: 

In [16]:
print("Valor objetivo (MX):", pe.value(model.MX))
for t in model.t:
    print(
        f"{t:>2s}  s_m1={pe.value(model.S[t,'m1']):5.1f}   "
        f"s_m2={pe.value(model.S[t,'m2']):5.1f}   "
        f"delay={pe.value(model.P[t]):5.1f}"
    )
print("END =", pe.value(model.END))


Valor objetivo (MX): 9.0
T1  s_m1= 23.0   s_m2= 32.0   delay=  9.0
T2  s_m1= 17.0   s_m2= 24.0   delay=  9.0
T3  s_m1=  7.0   s_m2= 17.0   delay=  9.0
T4  s_m1= 32.0   s_m2= 44.0   delay=  9.0
T5  s_m1= -0.0   s_m2=  7.0   delay=  9.0
END = 54.0


#### Solving problem 1 : 

In [17]:
model.obj = pe.Objective(expr=model.END, sense=pe.minimize) 
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee=True) 

print("Status:", result.solver.status)
print("Termination:", result.solver.termination_condition)

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.AbstractScalarObjective'>).
This is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
Set parameter Username
Set parameter LicenseID to value 2707518
Academic license - for non-commercial use only - expires 2026-09-11
Read LP format model from file C:\Users\Guill\AppData\Local\Temp\tmprbkejkim.pyomo.lp
Reading time = 0.05 seconds
x1: 122 rows, 57 columns, 284 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 122 rows, 57 columns and 284 nonzeros
Model fingerprint: 0xfc7f5107
Variable types: 0 continuous, 57 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9

In [18]:
print("Valor objetivo (MX):", pe.value(model.MX))
for t in model.t:
    print(
        f"{t:>2s}  s_m1={pe.value(model.S[t,'m1']):5.1f}   "
        f"s_m2={pe.value(model.S[t,'m2']):5.1f}   "
        f"delay={pe.value(model.P[t]):5.1f}"
    )
print("END =", pe.value(model.END))

Valor objetivo (MX): 16.0
T1  s_m1= 31.0   s_m2= 41.0   delay= 13.0
T2  s_m1= 25.0   s_m2= 33.0   delay= 16.0
T3  s_m1= 15.0   s_m2= 26.0   delay=  0.0
T4  s_m1=  7.0   s_m2= 16.0   delay= -0.0
T5  s_m1= -0.0   s_m2=  7.0   delay= -0.0
END = 53.0
